## Linear Elasticity for Cantilever Beam

In [5]:
using Gridap, Gridap.CellData, SparseArrays, BenchmarkTools
using NearestNeighbors, Printf  # make sure this is at the top of your file
using Makie,GLMakie, LaTeXStrings, Makie.GeometryBasics
using Gridap
using GridapGmsh

### Add labels to left and right faces for boundary conditions

In [6]:
model = GmshDiscreteModel("Lbracket2d_Sy.msh")


Info    : Reading 'Lbracket2d_Sy.msh'...
Info    : 15 entities
Info    : 36229 nodes
Info    : 36635 elements
Info    : Done reading 'Lbracket2d_Sy.msh'


ErrorException: 

### Construct weak form in Gridap.jl

In [26]:
# Integrating on the domain Ω
degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω, degree)
# neumann boundaries
Γ = BoundaryTriangulation(model,tags = "Load")
dΓ = Measure(Γ,degree)
# Vector-valued FE space
order = 1
reffe = ReferenceFE(lagrangian, VectorValue{2,Float64}, order)

V = TestFESpace(Ω, reffe; conformity=:H1, dirichlet_tags = "Top")
U = TrialFESpace(V)

# Constitutive law for LINEAR ELASTICITY 
const ν = 0.3   # Poisson's ratio
const E = 119e3 # Young's modulus

const λ = (E*ν)/((1+ν)*(1-2*ν)) # C12
const μ = E/(2*(1+ν)) # Shear modulus
ε₀(u) = 0.5 * ( ∇(u) + transpose(∇(u))) # Strain; In Gridap this can be automatically defined in Gridap.ε
σ(ε₀) = λ * tr(ε₀) * one(ε₀) + 2μ * ε₀; # Stress

## Run finite element analysis

In [27]:
# The weak form
F(x) = VectorValue(0.0, -50) # A unit load along Y axis applies to each node on the right face
a(u,v) = ∫((σ∘ε₀(u)) ⊙ ε₀(v)) *dΩ # Left-hand size; (∘) Composite functions
l(v) = ∫(F ⋅ v) * dΓ # Right-hand size
# Solution of the FE problem
op = AffineFEOperator(a,l,U,V)
uh = solve(op)
# writevtk(Ω,"L_bracket_uh",cellfields=["uh"=>uh,"epsi"=>ε₀(uh),"sigma"=>σ∘ε₀(uh)])

SingleFieldFEFunction():
 num_cells: 35828
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 10711105065389535731

### Compute the compliance

In [28]:
# Method 1
C1 = sum(∫((σ ∘ ε₀(uh)) ⊙ ε₀(uh)) * dΩ) # Compliance
println("Compliance 1: ", C1)
# Method 2
C2 = sum(∫(F ⋅ uh) * dΓ) # Compliance
println("Compliance 2: ", C2)
# Method 3
U = get_free_dof_values(uh) # displacement vector
K = get_matrix(op) # stiffness matrix
C3 = dot(U, K * U)
println("Compliance 3: ", C3)
# Method 4 using for loop to compute sum(ue^T*ke*ue)

using Gridap.Arrays, Gridap.FESpaces

cell_dofs = get_cell_dof_values(uh)
u = get_trial_fe_basis(V)
v = get_fe_basis(V)
K_e = get_array(∫((σ ∘ ε₀(u)) ⊙ ε₀(v)) * dΩ)
numele = num_cells(Ω) # Number of elements
C4 = 0.0 # Comliance
for e=1:numele
    ue = cell_dofs[e]    # element displacement
    ke = K_e[e] # element stiffness matrix
    C4 = C4 + transpose(ue)*ke*ue
end
println("Compliance 4: ", C4)

Compliance 1: 57.8758596405091
Compliance 2: 57.875859640437945
Compliance 3: 57.87585964028993
Compliance 4: 57.87585964045955
